In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
import ast
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from redditqa.dataset import load_reddit_dataset
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
import os
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import (
    pipeline,
    TextGenerationPipeline,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSequenceClassification,
    GPTNeoXForCausalLM,
    LlamaForSequenceClassification,
)
from redditqa.dataset import load_reddit_dataset
from transformers.utils.hub import convert_file_size_to_int, get_checkpoint_shard_files
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    set_peft_model_state_dict,
)
from peft.utils import _get_submodules
import peft
import torch
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats import inter_rater as irr
import base64
from collections import defaultdict
import textstat

/workspaces/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-21 11:17:01.042295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 11:17:01.701415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
torch.manual_seed(42)

#### Loading the eval dataset

In [4]:
eval_dataset = load_reddit_dataset("eval")
eval_dataset = eval_dataset.select(range(200))

Loading cached processed dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-cba55e4212677d14.arrow
Loading cached shuffled indices for dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-71c7c533e04253a7.arrow


In [5]:
questions = eval_dataset['question_title']

#### Run models

In [6]:
model_checkpoints = {
    # 7B Chat PPO
    '7B-chat_ppo': '/scratch1/jhoff/checkpoints/ppo_llama-2-7b_sft-checkpoint-2500/final_merged',
    # 3B
    '3B_baseline': 'openlm-research/open_llama_3b_v2',
    '3B_cp-2500': '/scratch1/jhoff/checkpoints/finetuned_open-llama-3b-v2/checkpoint-2500_merged',
    # 7B
    '7B_baseline': 'meta-llama/Llama-2-7b-hf',
    '7B_cp-2000': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-hf/checkpoint-2000_merged',
    # 7B Chat
    '7B-chat_baseline': 'meta-llama/Llama-2-7b-chat-hf',
    '7B-chat_cp-2500': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-2500_merged',
}

Generate answers for questions with the models

In [7]:
template = "<|ELIF|> Question: %question\nAnswer: "

def generate_answers(model_path):

    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.config.pad_token_id = model.config.eos_token_id
    model = model.cuda().eval()

    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "min_length": 32,
        "max_length": 128,
    }

    pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device='cuda:0')

    answers = {}

    for question in tqdm(questions):
        prompt = template.replace('%question', question)
        result = pipeline(prompt, **generation_kwargs, return_full_text=False)
        result = result[0]['generated_text']

        answers[question] = result

    return answers


model_answers = {}
for model_name, checkpoint in model_checkpoints.items():
    model_answers[model_name] = generate_answers(checkpoint)

  5%|▌         | 10/200 [00:11<03:01,  1.04it/s]/workspaces/reddit_qa/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1082: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 200/200 [03:46<00:00,  1.13s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.
  0%|          | 0/200 [00:00<?, ?it/s]/workspaces/reddit_qa/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a dep

In [8]:
{model_name: list(answers.values())[:3] for model_name, answers in model_answers.items()}

{'7B-chat-ppo': ["1. It's not that easy to start a town.  You have to have a well and a water supply, which can be expensive.  You have to have a sewage system.  You have to have roads.  You have to have a place for people to work.  You have to have a place for people to live. ",
  '80% of the time they are just used to deter crime, not to solve any particular crime.',
  "1. He's not giving away most of his money.  He's giving away a small fraction of his money.\n\n2. He's not giving away most of his money to individuals.  He's giving it away to charitable organizations."],
 '3B_baseline': ['13 Oct, 2008, 12:58 am: How is a skyscrapper made?\nand many other questions.\nThey also have the Open Radio channel.',
  '8P vs 1080p\n6C "Why Boundaries?"\n6D Separate electrical connections for every light in a room, for example for the family bathroom in the basement.\n6E The network connection to a computer in a server room or in a closet in an office.\n6F Cloud Computing, Wikipedia.\nQUESTION

#### Reward scores

Now, let's run our reward model on these generations

In [9]:
# Load the reward model
reward_model_checkpoint = "/scratch1/jhoff/checkpoints/reward_llama-2-7b-chat-hf/checkpoint-3000_merged"

tokenizer = AutoTokenizer.from_pretrained(reward_model_checkpoint)

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_checkpoint, 
    num_labels=1, 
    torch_dtype=torch.bfloat16
)

reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device=0,
)

Loading checkpoint shards: 100%|██████████| 3/3 [03:24<00:00, 68.31s/it]


In [10]:
# Apply the reward model to the answers 
template = "<|ELIF|> Question: %question\nAnswer: %answer"

model_rewards = {}
for model_name, model_qas in model_answers.items(): 
    print(f"Applying reward model to answers from {model_name}")
    model_rewards[model_name] = {}
    for question, answer in tqdm(model_qas.items()):
        # Create prompt based on the question and answer
        prompt = template.replace("%question", question).replace("%answer", answer)

        # Apply the reward model to the prompt
        reward = reward_pipe(prompt)[0]["score"]

        # Store the reward
        model_rewards[model_name][question] = reward

Applying reward model to answers from 7B-chat-ppo


100%|██████████| 200/200 [00:07<00:00, 27.83it/s]


Applying reward model to answers from 3B_baseline


100%|██████████| 200/200 [00:10<00:00, 19.41it/s]


Applying reward model to answers from 3B_cp-2500


...

100%|██████████| 200/200 [00:08<00:00, 22.97it/s]


Applying reward model to answers from 7B-chat_baseline


100%|██████████| 200/200 [00:10<00:00, 18.48it/s]


Applying reward model to answers from 7B-chat_cp-2500


100%|██████████| 200/200 [00:08<00:00, 22.52it/s]


Create a dataframe containing the Answers

In [11]:
dfs = {}

for name, qas in model_answers.items():

    rewards = model_rewards[name]

    df = pd.DataFrame.from_records([
        {'question': q, f'{name}.answer': a, f'{name}.reward': rewards[q]} 
        for q, a in qas.items()
    ])

    dfs[name] = df

first_df_name = list(dfs.keys())[0]
df = dfs[first_df_name]
for name, df_other in dfs.items():
    
    if name == first_df_name:
        continue

    df = df.merge(df_other, on='question')

df

,question,7B-chat-ppo.answer,7B-chat-ppo.reward,3B_baseline.answer,3B_baseline.reward,3B_cp-2500.answer,3B_cp-2500.reward,7B_baseline.answer,7B_baseline.reward,7B_cp-2000.answer,7B_cp-2000.reward,7B-chat_baseline.answer,7B-chat_baseline.reward,7B-chat_cp-2500.answer,7B-chat_cp-2500.reward
0,How does new city formation works in US? If me...,1. It's not that easy to start a town. You ha...,0.908618,"13 Oct, 2008, 12:58 am: How is a skyscrapper m...",0.767690,it's county by default. the boundaries are def...,0.844922,1. No. 2. No. 3. No. 4. No. 5. No. 6. No. 7. N...,0.846958,50 people can't form their own city.\n\nIt tak...,0.939913,|<|ELIF|> Question: How does new city formati...,0.946597,1. It's not just land. You need to have a basi...,0.971564
1,why aren't CCTV cameras made to produce higher...,80% of the time they are just used to deter cr...,0.875787,"8P vs 1080p\n6C ""Why Boundaries?""\n6D Separate...",0.823328,1) The third party article is not credible. It...,0.852935,1. The cost would be prohibitive. 2. The stora...,0.893309,1) The cost of the camera.\n\n2) The cost of t...,0.933431,There are several reasons why CCTV cameras ar...,0.976669,1. They are already pretty good quality.\n\n2....,0.953966
2,how is Bill Gates still so rich if he is givin...,1. He's not giving away most of his money. He...,0.944177,1.) He does not want to give away his money be...,0.525247,1) The $30B he gave away from the Bill &amp; M...,0.876634,1. Bill Gates has not given away most of his m...,0.962108,1. He still owns a huge chunk of Microsoft. H...,0.942507,Bill Gates is still rich because he has a sig...,0.957912,1) He is still the CEO of Microsoft.\n\n2) He ...,0.913677
3,How did America become so powerful despite bei...,1. We had the most powerful army in the world ...,0.901921,1) Highest development in society: Legalism vs...,0.554712,1. Population growth: we're growing faster tha...,0.781331,"1) The US was basically a British colony, and ...",0.955981,1. They have a lot of resources.\n2. They have...,0.931463,There are several reasons why America became ...,0.961534,1) Industrial revolution - they were able to p...,0.939025
4,How do they make white gold? Can it be melted ...,White gold is made by mixing 90% of gold with...,0.921922,900 parts gold to 100 parts other metals with ...,0.879974,"\n\nYes, these kinds of solutions trickle down...",0.867934,1) White gold is not pure gold. It is an alloy...,0.961534,"14k gold is 58.3% gold and 41.7% other stuff, ...",0.968856,White gold is an alloy of gold that has been ...,0.971564,1) White gold is made by coating a yellow gold...,0.979354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"In a roll of duct tape, how does the glue stic...",1. The adhesive is applied to the paper backin...,0.953966,11. |\nQuestion: An employee picks out a numbe...,0.779993,1. Duct tape works because of air bubbles caus...,0.954647,34% of the time.\n<|ELIF|> Question: If you ha...,0.918480,1. The adhesive is applied to a thin strip of ...,0.975946,The glue in a roll of duct tape is actually o...,0.959762,1. The glue is on the back of the tape\n2. The...,0.952574
196,How do popular social media apps become popula...,1. Have a simple and attractive UI/UX. People ...,0.874935,"\n<|ELIF|> Question: Who are the best backers,...",0.908618,1. Viral Explosion\n2. Advertise some on mains...,0.735642,0 is an arbitrary number. The idea is that onc...,0.835484,1. You have to have a large initial user base....,0.931463,0<|ELIF|> Popular social media apps often star...,0.971129,1) They are free and people are willing to try...,0.941654
197,Are we fishing at too high of a rate? Are we g...,1) We are moving into new waters and fishing i...,0.879974,96% of the human race has replaced fishing wit...,0.789234,"90% of what the US catches, is thrown back in ...",0.819893,95% of the fish in the ocean are in the top 5%...,0.918480,\nHumans will probably run out of fish before ...,0.931463,"|<|ELIF|>| No, we're not fish

Show mean reward scores for different checkpoints

In [12]:
reward_cols = [c for c in df.columns if c.endswith('.reward')]
reward_mean = df[reward_cols].mean()
reward_std = df[reward_cols].std()
reward_str = reward_mean.apply(lambda x: f'{x:.4f}').str.cat(reward_std.apply(lambda x: f' ± {x:.4f}'))
reward_str

7B-chat-ppo.reward         0.9037 ± 0.0588
3B_baseline.reward         0.8240 ± 0.0778
3B_cp-2500.reward          0.8271 ± 0.0928
7B_baseline.reward         0.9103 ± 0.0533
7B_cp-2000.reward          0.9163 ± 0.0562
7B-chat_baseline.reward    0.9659 ± 0.0175
7B-chat_cp-2500.reward     0.9201 ± 0.0563
dtype: object

#### Textstat and toxicity scores

In [40]:
toxicity_pipe = pipeline("text-classification", model="tomh/toxigen_roberta")


def mean_std(col, precision=2): 
    col_mean = col.mean()
    col_std = col.std()

    return f'{col_mean:.{precision}f} ± {col_std:.{precision}f}'



def run_toxicity_pipe(text):
    try: 
        result = toxicity_pipe(text, top_k=None)
        result = [r for r in result if r['label'] == 'LABEL_1'][0]
    except:
        return 0.5

    return result['score']


metrics_list = []


for col in df.columns:

    if col == 'question':
        continue

    if not col.endswith('.answer'):
        continue

    name = col.replace('.answer', '')

    # Apply textstat functions
    text_standard = df[col].apply(lambda t: textstat.text_standard(t, float_output=True))
    reading_time = df[col].apply(textstat.reading_time)
    df[name + '.text_standard'] = text_standard
    df[name + '.reading_time'] = reading_time

    # Measure toxicity
    toxicity = df[col].apply(run_toxicity_pipe)

    # Save metrics
    metrics_list.append({
        'name': name,
        'text_standard [grade]': mean_std(text_standard),
        'reading_time [s]': mean_std(reading_time),
        'toxicity [0-1]': mean_std(toxicity, precision=3),
        'reward [0-1]': mean_std(df[name + '.reward'], precision=3),
    })

    # Print metrics
    print(f'{name}:')
    print('\n'.join([f'  {k}: {v}' for k, v in metrics_list[-1].items()]))


7B-chat-ppo:
  name: 7B-chat-ppo
  text_standard [grade]: 7.00 ± 3.06
  reading_time [s]: 2.28 ± 1.28
  toxicity [0-1]: 0.041 ± 0.153
  reward [0-1]: 0.904 ± 0.059
3B_baseline:
  name: 3B_baseline
  text_standard [grade]: 10.14 ± 5.42
  reading_time [s]: 3.84 ± 1.05
  toxicity [0-1]: 0.047 ± 0.161
  reward [0-1]: 0.824 ± 0.078
3B_cp-2500:
  name: 3B_cp-2500
  text_standard [grade]: 8.77 ± 4.15
  reading_time [s]: 3.21 ± 1.58
  toxicity [0-1]: 0.055 ± 0.160
  reward [0-1]: 0.827 ± 0.093
7B_baseline:
  name: 7B_baseline
  text_standard [grade]: 6.78 ± 3.24
  reading_time [s]: 3.74 ± 0.78
  toxicity [0-1]: 0.068 ± 0.188
  reward [0-1]: 0.910 ± 0.053
7B_cp-2000:
  name: 7B_cp-2000
  text_standard [grade]: 6.33 ± 3.29
  reading_time [s]: 2.99 ± 1.13
  toxicity [0-1]: 0.047 ± 0.174
  reward [0-1]: 0.916 ± 0.056
7B-chat_baseline:
  name: 7B-chat_baseline
  text_standard [grade]: 11.21 ± 2.82
  reading_time [s]: 4.74 ± 0.82
  toxicity [0-1]: 0.002 ± 0.008
  reward [0-1]: 0.966 ± 0.018
7B-chat_

In [41]:
metrics_list[0]['name'] = '7B-chat_ppo'

In [49]:
metrics_df = pd.DataFrame(metrics_list)
metrics_df = metrics_df.set_index('name').sort_index()
metrics_df.to_markdown('metrics_model_comparison.md')
metrics_df

,text_standard [grade],reading_time [s],toxicity [0-1],reward [0-1]
name,,,,
3B_baseline,10.14 ± 5.42,3.84 ± 1.05,0.047 ± 0.161,0.824 ± 0.078
3B_cp-2500,8.77 ± 4.15,3.21 ± 1.58,0.055 ± 0.160,0.827 ± 0.093
7B-chat_baseline,11.21 ± 2.82,4.74 ± 0.82,0.002 ± 0.008,0.966 ± 0.018
7B-chat_cp-2500,5.92 ± 2.66,3.07 ± 1.05,0.034 ± 0.119,0.920 ± 0.056
7B-chat_ppo,7.00 ± 3.06,2.28 ± 1.28,0.041 ± 0.153,0.904 ± 0.059
7B_baseline,6.78 ± 3.24,3.74 ± 0.78,0.068 ± 0.188,0.910 ± 0.053
7B_cp-2000,6.33 ± 3.29,2.99 ± 1.13,0.047 ± 0.174,0.916 ± 0.056


Review an example row

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(pd.DataFrame(df.iloc[0]))

,0
question,"How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?"
7B-chat-ppo.answer,"1. It's not that easy to start a town. You have to have a well and a water supply, which can be expensive. You have to have a sewage system. You have to have roads. You have to have a place for people to work. You have to have a place for people to live."
7B-chat-ppo.reward,0.908618
3B_baseline.answer,"13 Oct, 2008, 12:58 am: How is a skyscrapper made?\nand many other questions.\nThey also have the Open Radio channel."
3B_baseline.reward,0.76769
3B_cp-2500.answer,"it's county by default. the boundaries are defined by a formula historically (think easy rider, also detroit industrial county) used."
3B_cp-2500.reward,0.844922
7B_baseline.answer,1. No. 2. No. 3. No. 4. No. 5. No. 6. No. 7. No. 8. No. 9. No. 10. No. 11. No. 12. No. 13. No. 14. No
7B_baseline.reward,0.846958
7B_cp-2000.answer,"50 people can't form their own city.\n\nIt takes 1000 people.\n\nWhen you meet the 1000 people requirement, you get a charter.\n\nThat charter is a document that says you get to form a city, but you still have to follow all the state rules."


#### Data for Human Annotators 

Generate sheet to compare models for human annotators. We focus on the baseline and the two checkpoints after 2500 and 8500 steps. We compare to the left and make sure that the order is randomized

In [43]:
def encode_metadata(metadata):
    metadata = repr(metadata)
    metadata = metadata.encode('utf-8')
    metadata = base64.b64encode(metadata)
    return metadata

def decode_metadata(metadata):
    metadata = metadata[2:-1]  # remove b' and '
    metadata = base64.b64decode(metadata)
    metadata = metadata.decode('utf-8')
    metadata = ast.literal_eval(metadata)
    return metadata

In [44]:
def create_comparison(col1, col2): 

    # Create a new dataframe with the two columns
    comparisons = []

    for i, row in df.iterrows():

        question = row['question']
        answer1 = row[col1]
        answer2 = row[col2]

        do_swap = np.random.choice([True, False])


        comparison_row = {
            'question': question,
            'answer1': answer1 if not do_swap else answer2,
            'answer2': answer2 if not do_swap else answer1,
            'metadata': {
                'rand1': np.random.randint(0, 1e5),
                'do_swap': do_swap,
                'answer1': col1 if not do_swap else col2,
                'answer2': col2 if not do_swap else col1, 
                'rand2': np.random.randint(0, 1e5)
            },
        }
        comparison_row['metadata_encoded'] = encode_metadata(comparison_row['metadata'])

        comparisons.append(comparison_row)

    df_comparisons = pd.DataFrame.from_records(comparisons)
    return df_comparisons

#with pd.option_context('display.max_colwidth', None):
#    display(create_comparison('baseline.answer', 'checkpoint-2500.answer'))

In [45]:
df.columns

Index(['question', '7B-chat-ppo.answer', '7B-chat-ppo.reward',
       '3B_baseline.answer', '3B_baseline.reward', '3B_cp-2500.answer',
       '3B_cp-2500.reward', '7B_baseline.answer', '7B_baseline.reward',
       '7B_cp-2000.answer', '7B_cp-2000.reward', '7B-chat_baseline.answer',
       '7B-chat_baseline.reward', '7B-chat_cp-2500.answer',
       '7B-chat_cp-2500.reward', '7B-chat-ppo.text_standard',
       '7B-chat-ppo.reading_time', '3B_baseline.text_standard',
       '3B_baseline.reading_time', '3B_cp-2500.text_standard',
       '3B_cp-2500.reading_time', '7B_baseline.text_standard',
       '7B_baseline.reading_time', '7B_cp-2000.text_standard',
       '7B_cp-2000.reading_time', '7B-chat_baseline.text_standard',
       '7B-chat_baseline.reading_time', '7B-chat_cp-2500.text_standard',
       '7B-chat_cp-2500.reading_time'],
      dtype='object')

In [46]:
# Yellow ones
create_comparison('3B_baseline.answer', '3B_cp-2500.answer')\
    .head(100)\
    .to_excel('Model Comparison - 3B Baseline vs 3B SFT (n100).xlsx')

create_comparison('7B_baseline.answer', '7B_cp-2000.answer')\
    .head(100)\
    .to_excel('Model Comparison - 7B Baseline vs 7B SFT (n100).xlsx')

# Green ones
create_comparison('3B_cp-2500.answer', '7B_cp-2000.answer')\
    .head(100)\
    .to_excel('Model Comparison - 3B SFT vs 7B SFT (n100).xlsx')

create_comparison('7B_cp-2000.answer', '7B-chat_cp-2500.answer')\
    .head(100)\
    .to_excel('Model Comparison - 7B SFT vs 7B Chat SFT (n100).xlsx')

# RLHF
create_comparison('7B-chat-ppo.answer', '7B-chat_cp-2500.answer')\
    .head(100)\
    .to_excel('Model Comparison - 7B PPO vs 7B Chat SFT (n100).xlsx')

create_comparison('7B-chat-ppo.answer', '7B-chat_baseline.answer')\
    .head(100)\
    .to_excel('Model Comparison - 7B PPO vs 7B Chat Baseline (n100).xlsx')


# create_comparison('baseline.answer', 'checkpoint-2500.answer')\
#     .to_excel('Model Comparison - Baseline vs Checkpoint-2500 (n200).xlsx')
#
# create_comparison('checkpoint-2500.answer', 'checkpoint-8500.answer')\
#     .to_excel('Model Comparison - Checkpoint-2500 vs Checkpoint-8500 (n200).xlsx')

#### EDA on Data from Human Annotators

First, let's count the votes

In [52]:
comparison_files = [
    'labelled_data/Model Comparison - Baseline vs Checkpoint-2500 (n200).xlsx',
    'labelled_data/Model Comparison - Checkpoint-2500 vs Checkpoint-8500 (n200).xlsx',
    'labelled_data/Model Comparison - 3B Baseline vs 3B SFT (n100).xlsx',
    'labelled_data/Model Comparison - 7B Baseline vs 7B SFT (n100).xlsx',
]


def organize_answers(row): 
    metadata = row['metadata']
    answer1 = metadata['answer1']
    answer2 = metadata['answer2']

    return {answer1: row['answer1'], answer2: row['answer2']}


answers = {}
for comparison_file in comparison_files:

    # Load comparison data
    comparison_df = pd.read_excel(comparison_file)

    # Decode metadata
    comparison_df["metadata"] = comparison_df.metadata_encoded.apply(decode_metadata)
    comparison_df.drop(columns=["metadata_encoded"], inplace=True)
    comparison_df.metadata = comparison_df.metadata.apply(lambda d: {k: v for k, v in d.items() if k not in ['rand1', 'rand2']})

    # Fix column names
    comparison_df.drop(columns=["assignee1"], inplace=True, errors="ignore")
    comparison_df.rename(columns={"label1": "label"}, inplace=True, errors="ignore")

    # Count votes
    counters = defaultdict(int)
    for _, row in comparison_df.iterrows():
        if row['label'] == '1 - answer1 is better':
            counters[row['metadata']['answer1']] += 1
        
        elif row['label'] == '2 - answer2 is better':
            counters[row['metadata']['answer2']] += 1

        elif row['label'] == '3 - both are equal':
            counters['parity'] += 1

    # Organize answers
    answers_pair = comparison_df.apply(organize_answers, axis=1)
    answers_pair = list(answers_pair)
    model_names = {k for pair in answers_pair for k in pair.keys() }
    for model_name in model_names: 
        answers[model_name] = [pair[model_name] for pair in answers_pair]

    #print(answers_pair[:2])

    counters = {name: counters[name] for name in sorted(counters.keys())}
    print(dict(counters))


# # Create df with answers
# answers = pd.DataFrame(answers)
# #answers.columns = [c.replace('.answer', '') for c in answers.columns]
# answers.insert(0, "question", comparison_df["question"])
#
#
# # Display
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
#     display(answers.head(3))

{'baseline.answer': 78, 'checkpoint-2500.answer': 40, 'parity': 33}
{'checkpoint-2500.answer': 68, 'checkpoint-8500.answer': 60, 'parity': 46}
{'3B_baseline.answer': 16, '3B_cp-2500.answer': 53, 'parity': 31}
{'7B_baseline.answer': 24, '7B_cp-2000.answer': 60, 'parity': 16}
